## Code Setup

In [ ]:
import os
import matplotlib.pyplot as plt
import os, sys
sys.path.append(os.path.join(os.getcwd(), "birdmaps"))
import bird_runs

In [ ]:
FILE_PATH = "data/CA-EcoScape-Paper/"
bird_run = bird_runs.BirdRun(FILE_PATH)

In [ ]:
TRANSMISSION_EXPONENT = 1
RUN_NAME = "Jun9-gradient"

In [ ]:
acowoo_run = bird_run.get_bird_run("acowoo", "Acorn Woodpecker", 
                                   transmission_exponent=TRANSMISSION_EXPONENT, run_name=RUN_NAME)
stejay_run = bird_run.get_bird_run("stejay", "Steller's Jay", 
                                   transmission_exponent=TRANSMISSION_EXPONENT, run_name=RUN_NAME)
oaktit_run = bird_run.get_bird_run("oaktit308", "Oak Titmouse", 
                                   transmission_exponent=TRANSMISSION_EXPONENT, run_name=RUN_NAME)
birds = [acowoo_run, stejay_run, oaktit_run]


In [ ]:
import csv, json

def read_resistance_csv(fn):
    """Reads a dictionary of terrain to resistance in csv, producing a dictionary."""
    d = {}
    with open(fn) as f:
        reader = csv.DictReader(f)
        for row in reader:
            d[int(row['map_code'])] = float(row['resistance'])
    return d

def read_histogram_json(fn):
    with open(fn) as f:
        d = json.load(f)
        return {int(k): v for k, v in d.items()}

def write_transmission_csv(resistance, fn):
    """
    :resistance: a dictionary mapping map codes (integers) to resistance values (float)"""
    with open(fn, "w") as f:
        writer = csv.DictWriter(f, fieldnames=['map_code', 'transmission'])
        writer.writeheader()
        for k, v in resistance.items():
            writer.writerow({'map_code': k, 'transmission': v})

def plot_histogram(d, title=""):
    xs = list(d.keys())
    xs.sort(reverse=True)
    sxs = [str(x) for x in xs]
    ys = [d[x] for x in xs]
    plt.barh(sxs, ys, label=sxs)
    plt.title(title)
    plt.show()

In [ ]:
# See https://docs.google.com/spreadsheets/d/13pFa8tlHpjWYT0t_HHOE7LoZPoOPaBvO/edit#gid=1586882738
def resistance_generic_terrain(t):
    if 100 <= t < 200:
        return 0 # Forest
    elif 200 <= t < 300:
        return 0.4 # Savanna
    elif 300 <= t < 400:
        return 0. # Shrubland
    elif 400 <= t < 500: 
        return 0.2 # Grassland
    elif 500 <= t < 600:
        return 0.3 # Wetlands
    elif 600 <= t < 700: 
        return 0.3 # Rocky
    elif 700 <= t < 800:
        return 0.4 # Caves
    elif 800 <= t < 900:
        return 0.4 # Desert
    elif t == 1401:
        return 0.6
    elif t == 1402: 
        return 0. # This is very good territory for flyover. 
    elif t in [1403, 1404]:
        return 0.2
    elif t == 1405: # Urban
        return 0.6
    elif t == 1406:
        return 0
    elif 1507 <= t <= 1510:
        return 0.6 
    elif t == 1600:
        return 0.2
    else:
        return 1.

In [ ]:
def merge_transmission_forest_birds(resistance_d):
    """Merges the resistance histogram with the observation dictionary."""
    # These are all the terrains.
    terrains = list(resistance_d.keys())
    terrains.sort()
    # I initialize the transmission using the resistance dictionary.
    transmission = {}
    for t in terrains:
        # Computes the resistance. 
        r = min(resistance_d.get(t, 1.), resistance_generic_terrain(t))
        transmission[t] = (1 - r) ** TRANSMISSION_EXPONENT
    return transmission


In [ ]:
from collections import defaultdict
import matplotlib

matplotlib.rc('figure', figsize=(8, 8))

def compute_transmission(bird, show=False, use_generic=True):
    """Diagnoses the terrain for a bird, and computes the transmission"""
    resistance_d = read_resistance_csv(bird.resistance_original_fn)
    transmission = merge_transmission_forest_birds(resistance_d)
    write_transmission_csv(transmission, bird.transmission_fn)
    if show:
        plot_histogram(resistance_d, title="{} original resistance".format(bird.name))
        plot_histogram(transmission, title="{} transmission".format(bird.name))
    print("Done", bird.name)

In [ ]:
compute_transmission(acowoo_run, show=True)

In [ ]:
compute_transmission(oaktit_run, show=True)

In [ ]:
for b in birds:
    compute_transmission(b)